In [7]:
from glob import glob
import os
import pandas as pd

def get_unique_pids(dirpath):
    pids = sorted([get_patient_id(f) for f in glob('{}/*.nii.gz'.format(dirpath))])
    pids = sorted(list(set(pids)))
    return pids


def get_patient_id(fpath):
    fname = fpath.split('/')[-1].replace('.nii.gz', '')
    pid = '_'.join(fname.split('_')[:-1])
    return pid
    

t2_pids = get_unique_pids('/mnt/datasets/srivathsa/ADNI/full_brain')
fl_pids = get_unique_pids('/mnt/datasets/srivathsa/ADNI_FLAIR/full_brain')
all_pids = [pid for pid in fl_pids if pid in t2_pids]

In [18]:
print(len(all_pids))

21


In [8]:
df_adni = pd.read_csv('adni_cn.csv')

In [14]:
dict_adni = df_adni.to_dict(orient='records')

subj_map = {}

for drow in dict_adni:
    if drow['Subject ID'] not in subj_map:
        subj_map[drow['Subject ID']] = {
            'age': drow['Age'],
            'sex': drow['Sex'],
            'series_list': [drow['Description']]
        }
    else:
        subj_map[drow['Subject ID']]['series_list'].append(drow['Description'])

for k, v in subj_map.items():
    subj_map[k]['series_list'] = list(set(subj_map[k]['series_list']))

In [20]:
filt_cases = []

for k, v in subj_map.items():
    has_mprage = len([s for s in v['series_list'] if 'mprage' in s.lower().replace('-', '')]) > 0
    has_t2 = len([s for s in v['series_list'] if 't2 fse' in s.lower()]) > 0
    has_fl = len([s for s in v['series_list'] if '3d flair' in s.lower()]) > 0
    
    if has_t2 and has_fl:
        filt_cases.append(k)

print(len(filt_cases))

37
